<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/dev-branch/featuretools_and_TPOT_workflow_ver_0p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import featuretools as ft
import sklearn as sk
import featuretools as ft

#Reading Data

##From a Local Drive

In [0]:
#from google.colab import files
#uploaded = files.upload()

#import io
#df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))

##From Google Drive via PyDrive

In [2]:
# Go to the CSV file in Google Drive, right-click on it and select 
# “Get shareable link”. The link will be copied into your clipboard. 
# Paste this link into following a string variable i.e. link:

link = 'https://drive.google.com/open?id=10JNCo2Opquuq2cwGiyGgvlds8U2wz3mW'  # The shareable link

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


fluff, id = link.split('=')   # Verify that you have everything after '=' by using print(id)

# Finally reading the data into a dataframe

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dummy_file_name.csv')  
df = pd.read_csv('dummy_file_name.csv')

    100% |████████████████████████████████| 993kB 4.0MB/s 


In [3]:
df.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [0]:
target = df.Creditability
df = df.drop(columns = target.name)
df['df_index'] = range(0,len(df))

In [4]:
df.columns

Index(['Account Balance', 'Duration of Credit (month)',
       'Payment Status of Previous Credit', 'Purpose', 'Credit Amount',
       'Value Savings/Stocks', 'Length of current employment',
       'Instalment per cent', 'Sex & Marital Status', 'Guarantors',
       'Duration in Current address', 'Most valuable available asset',
       'Age (years)', 'Concurrent Credits', 'Type of apartment',
       'No of Credits at this Bank', 'Occupation', 'No of dependents',
       'Telephone', 'Foreign Worker', 'df_index'],
      dtype='object')

#Feature Augmentation

In [0]:
es = ft.EntitySet("mydata")


# Create an entity from the dataframe df
# This dataframe must have an index col (and if required, a time index col) 
#      namely 'df_index' (and time_index = 'dt_time_col')

es = es.entity_from_dataframe(entity_id = 'my_entity_id', dataframe = df, index = 'df_index')

#, time_index = 'dt_time_col'



# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='my_entity_id', 
                                 trans_primitives=["add","subtract"],
                                 max_depth = 2
                                )
#trans_primitives = ["absolute", "cum_count","subtract","cum_sum","haversine","or","mod","characters","cum_max","multiply","add"]
# max_features = 100, max_depth = 2

#Pipeline Optimization

In [9]:
X, y = df, target

X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, random_state=1)

sk.model_selection.tra

AttributeError: ignored

In [10]:
sk.__version__

'0.20.2'

In [0]:
#feature_matrix = pd.read_csv('./example_data.csv', index_col=0)


#X = feature_matrix.drop('MEAN(invoices.MEAN(item_purchases.UnitPrice))', axis=1)
#y = feature_matrix['MEAN(invoices.MEAN(item_purchases.UnitPrice))']


#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

#tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)

#tpot.fit(X_train, y_train)


#y_hat = tpot.predict(X_test)
#print("R2 score:", sklearn.metrics.r2_score(y_test, y_hat))

In [0]:
tpot.export('./tpot_pipeline.py')

#from the python file 'tpot_pipeline.py' copy following two lines verbatim

training_features, testing_features, training_target, testing_target = \
    train_test_split(features, tpot_data['class'], random_state=42)

exported_pipeline = make_pipeline(
    RobustScaler(),
    LinearSVR(C=15.0, dual=False, epsilon=1.0, 
              loss="squared_epsilon_insensitive", tol=0.0001)
)


exported_pipeline.fit(training_features, training_target)


In [0]:
#Show the features with each used coefficient:

important_coefs = pd.Series(data=exported_pipeline.steps[1][1].coef_, 
                            index=X.columns)

sorted_coef = important_coefs.sort_values(ascending=False)

sorted_coef